In [17]:
import os
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Weaviate
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader, PDFMinerLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

In [35]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map="cpu", 
    offload_folder="save_folder"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [11]:
SOURCE_DIRECTORY = "source_docs/8-tsne.pdf"
WEAVIATE_URL = "https://exactas-guru-cluster-l99i5920.weaviate.network"

In [12]:
print(f"Loading documents from {SOURCE_DIRECTORY}")
loader = PDFMinerLoader(SOURCE_DIRECTORY)
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(document)
print(f"Split into {len(texts)} chunks of text")

Loading documents from source_docs/8-tsne.pdf
Split into 107 chunks of text


In [15]:
embeddings = HuggingFaceInstructEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

load INSTRUCTOR_Transformer
max_seq_length  512


In [29]:
embeddings2 = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

In [30]:
db = Chroma.from_documents(texts, embeddings2, persist_directory="db")

In [16]:
import weaviate

client = weaviate.Client(
    url=WEAVIATE_URL, 
    auth_client_secret=weaviate.AuthApiKey(api_key="8Y9ZhRo1dVPjuXgsheQcZPd1DGaNtjTHvFaU")
)

In [18]:
vectorstore = Weaviate.from_documents(
    texts, embeddings, client=client, by_text=False
)

In [19]:
question = "What is the t-SNE algorithm?"

In [31]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

In [32]:
result_ = qa_chain(
    question
)
result = result_["result"].strip()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 result_ = qa_chain(                                                                          │
│   2 │   question                                                                                 │
│   3 )                                                                                            │
│   4 result = result_["result"].strip()                                                           │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/chains/base.py:310 in __call__                                                                │
│                                                                                                  │
│   307 │   │   │   )                                                                              │
│   308 │   │   except BaseException as e:                                                         │
│   309 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 310 │   │   │   raise e                                                                        │
│   311 │   │   run_manager.on_chain_end(outputs)                                                  │
│   312 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   313 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/chains/base.py:304 in __call__                                                                │
│                                                                                                  │
│   301 │   │   )                                                                                  │
│   302 │   │   try:                                                                               │
│   303 │   │   │   outputs = (                                                                    │
│ ❱ 304 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   305 │   │   │   │   if new_arg_supported                                                       │
│   306 │   │   │   │   else self._call(inputs)                                                    │
│   307 │   │   │   )                                                                              │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/chains/retrieval_qa/base.py:139 in _call                                                      │
│                                                                                                  │
│   136 │   │   │   docs = self._get_docs(question, run_manager=_run_manager)                      │
│   137 │   │   else:                                                                              │
│   138 │   │   │   docs = self._get_docs(question)  # type: ignore[call-arg]                      │
│ ❱ 139 │   │   answer = self.combine_documents_chain.run(                                         │
│   140 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   141 │   │   )                                                                                  │
│   142                                                      

In [21]:
template = """
[INST] <>
Act as a Machine Learning engineer who is teaching high school students.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [24]:
query = "Explain what the 't' in t-sne stands for in 2-3 sentences"
result = llm(prompt.format(text=query))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 query = "Explain what the 't' in t-sne stands for in 2-3 sentences"                          │
│ ❱ 2 result = llm(prompt.format(text=query))                                                      │
│   3                                                                                              │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/llms/base.py:876 in __call__                                                                  │
│                                                                                                  │
│    873 │   │   │   │   "`generate` instead."                                                     │
│    874 │   │   │   )                                                                             │
│    875 │   │   return (                                                                          │
│ ❱  876 │   │   │   self.generate(                                                                │
│    877 │   │   │   │   [prompt],                                                                 │
│    878 │   │   │   │   stop=stop,                                                                │
│    879 │   │   │   │   callbacks=callbacks,                                                      │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/llms/base.py:656 in generate                                                                  │
│                                                                                                  │
│    653 │   │   │   │   │   callback_managers, prompts, run_name_list                             │
│    654 │   │   │   │   )                                                                         │
│    655 │   │   │   ]                                                                             │
│ ❱  656 │   │   │   output = self._generate_helper(                                               │
│    657 │   │   │   │   prompts, stop, run_managers, bool(new_arg_supported), **kwargs            │
│    658 │   │   │   )                                                                             │
│    659 │   │   │   return output                                                                 │
│                                                                                                  │
│ /Users/juaniponce/Desktop/DocsJuan/Python/exactas-guru/.env/lib/python3.10/site-packages/langcha │
│ in/llms/base.py:544 in _generate_helper                                                          │
│                                                                                                  │
│    541 │   │   except BaseException as e:                                                        │
│    542 │   │   │   for run_manager in run_managers:                                              │
│    543 │   │   │   │   run_manager.on_llm_error(e)                                               │
│ ❱  544 │   │   │   raise e                                                                       │
│    545 │   │   flattened_outputs = output.flatten()                                              │
│    546 │   │   for manager, flattened_output in zip(run_managers, flattened_outputs):            │
│    547 │   │   │   manager.on_llm_end(flattened_output)                                          │
│                                                            